In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
#URL to wiki page
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# download wiki page 
wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [4]:
# use data attribute text to extract XML as a string and assign to page
page = (wiki_page.text)

In [7]:
# extracting data
beginning= page.find('<table class="wikitable sortable">')
end= page.find('</tbody></table>')

wiki_table_text = (page[beginning:end])
# print (wiki_table_text)

In [9]:


soup = BeautifulSoup(wiki_table_text,"html.parser")
table_rows = soup.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

#

df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df_filtered = df.query("Borough != 'Not assigned'")







In [10]:
#group by Postcode and Borough to make neighbourhood a list
def create_list(neighborhood_list):
    final_string = ""
    for neighbourhood in neighborhood_list:
        final_string = final_string + neighbourhood + ', '
    return final_string[:-2]

df_new = df_filtered.groupby(['Postcode', 'Borough']).agg({'Neighbourhood': create_list})
df_new

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
